# Window-функции

Используются для реорганизации и упрощения DML-запросов. Представляют собой "локальные" представления в рамках одного запроса.

Позволяют:
- разбить сложный запрос на логические, именованные блоки
- обращаться многократно к себе по имени из разных частей запроса
- организовывать рекурсивную обработку 

Синтаксис:

````
WITH  CTE_name [( column_name [ ,...n ] )]
AS
( 
    CTE_query_definition 
)
DML_Query
````

В качестве DML_Query может выступать запрос на основе любого из DML-операторов (SELECT, INSERT, DELETE, UPDATE, MERGE)

Пример вставки:


In [1]:
use SwimmingCompetitionOLTP
go

drop table if exists #PersonData
go

create table #PersonData
(
    id          int             not null identity primary key,
    FirstName   nvarchar(30)    not null, 
    LastName    nvarchar(30)    not null, 
    Birth       int             not null, 
    TeamCity    nvarchar(255) 
)
go

--вставляем данные в CTE!
with CTE_PersonData (FirstName, LastName, Birth, TeamCity) 
as
(
    select p.first_name, p.last_name, p.birth_year, t.name + ' ' + t.city
    from dbo.Person p
    left join dbo.Person_Team_xref xref on p.person_id = xref.person_id
    inner join dbo.Team t on xref.team_id = t.team_id
)
insert into #PersonData (FirstName, LastName, Birth, TeamCity) 
select FirstName, LastName, Birth, TeamCity from CTE_PersonData
go

select top 10 * from  #PersonData

Commands completed successfully.

Total execution time: 00:00:00.0780516

Commands completed successfully.

Total execution time: 00:00:00.0046077

Commands completed successfully.

...

(1301 rows affected)

Total execution time: 00:00:00.0193887

(10 rows affected)

Total execution time: 00:00:00.0879478

id,FirstName,LastName,Birth,TeamCity
1,Adomas,BEPIRŠTIS,2004,Impuls PA Vilnius
2,Anastasiya,YERMISHYNA,2006,Dolphin KIEV
3,Anton,KOCHU,2005,Dolphin KIEV
4,Anton,VORONA,2003,Dolphin KIEV
5,Daniil,TKACHENKO,2007,Dolphin KIEV
6,Daryana,ZHEREBINA,2006,Dolphin KIEV
7,Dominykas,ŠNAIDERIS,2007,Impuls PA Vilnius
8,Elita,VAŠKEVIČIŪTĖ,2005,Impuls PA Vilnius
9,Emilija,STEPONĖNAITĖ,2004,Impuls PA Vilnius
10,Enrika,KRAKELYTĖ,2008,Impuls PA Vilnius


Удаление из CTE:

In [2]:
use SwimmingCompetitionOLTP
go

drop table if exists #PersonData
go

create table #PersonData
(
    id          int             not null identity primary key,
    FirstName   nvarchar(30)    not null, 
    LastName    nvarchar(30)    not null, 
    Birth       int             not null
)
go

insert into #PersonData (FirstName, LastName, Birth)
values (N'Тимофей', N'Гавриленко', 1980),
        (N'Ольга', N'Гавриленко', 1980),
        (N'Илья', N'Гавриленко', 2006),
        (N'Тимофей', N'Гавриленко', 2007),
        (N'Анна', N'Гавриленко', 2009)
go

--удаляем данные из CTE!
with CTE_PersonData
as
(
    select 1 as fake_column
    from #PersonData
    where Birth < 2000
)
delete from CTE_PersonData
go

select * from  #PersonData

Commands completed successfully.

Total execution time: 00:00:00.0012788

Commands completed successfully.

Total execution time: 00:00:00.0029835

Commands completed successfully.

...

(2 rows affected)

Total execution time: 00:00:00.0134116

(3 rows affected)

Total execution time: 00:00:00.0099417

id,FirstName,LastName,Birth
3,Илья,Гавриленко,2006
4,Тимофей,Гавриленко,2007
5,Анна,Гавриленко,2009


<h1>Рекурсивные CTE</h1><br>
CTE, вызывающие сами себя.<br>
Состоят из двух или более <i>SELECT</i>-ов: анкорной части (блока инициализации) и рекурсивной части, объединенных с помощью <i>UNION ALL</i>.<br><br><i>Анкорная часть</i> выполняется один раз, <i>рекурсивная часть</i> повторяется многократно.<br><br>На каждой итерации к набору записей, уже имеющихся в CTE, добавляются новые записи. Итерации продолжаются до тех пор, пока приращение строк (дельта) не будет пустым множеством.<br><br>При обращении к CTE в рекурсивной части идет обращение не ко всем имеющимся записям, а только к последнему добавленному приращению.<br>Т.е., n-ый шаг зависит от (n-1)-ого шага.

In [4]:
with CTE_Recursive
as
(
    --анкорная часть
    select 1 as n
    union all
    --рекурсивная часть
    select n + 1 from CTE_Recursive where n + 1 <= 10
)
select * from CTE_Recursive

(10 rows affected)

Total execution time: 00:00:00.0039511

n
1
2
3
4
5
6
7
8
9
10


Глубина рекурсии по умолчанию не превосходит 100. В противном случае возникает исключение.<br>

In [7]:
with CTE_Recursive
as
(
    select 1 as n
    union all
    --нет условия остановки рекурсии
    select n + 1 from CTE_Recursive 
)
select * from CTE_Recursive

Обойти это ограничение можно с помощью опции запроса <i>(MAXRECURSION n)</i>. При n=0 ограничение снимается полностью, но нужно быть внимательным, т.к. возможно падение сервера из-за нехватки памяти.

In [8]:
with CTE_Recursive
as
(
    select 1 as n
    union all
    --нет условия остановки рекурсии
    select n + 1 from CTE_Recursive where n + 1 <= 1000
)
select * from CTE_Recursive
option(MAXRECURSION 1000)

<b>Примеры рекурсивных CTE</b>

Список букв английского алфавита:

In [3]:
;with CTE_Letters 
as(
    select ascii('a') code, char(ascii('a')) letter
    union all
    select code + 1, char(code + 1) 
	from CTE_Letters
    where code + 1 <= ascii('z')
)
select /*code,*/ letter from CTE_Letters

(26 rows affected)

Total execution time: 00:00:00.0269300

letter
a
b
c
d
e
f
g
h
i
j


Числа Фибоначчи до 1000:

In [1]:
;with Fibonacci(iter,a,b,c) 
as
(
 select iter=1, a=1, b=1, c=1+1
 union all
 select iter+1, a=b, b=c, c=b+c
 from Fibonacci 
 where b < 1000
)
select 'Fibonacci(' + cast(iter as varchar) + ') = ' + cast(a as varchar) as [output]
from Fibonacci

(16 rows affected)

Total execution time: 00:00:00.1286446

output
Fibonacci(1) = 1
Fibonacci(2) = 1
Fibonacci(3) = 2
Fibonacci(4) = 3
Fibonacci(5) = 5
Fibonacci(6) = 8
Fibonacci(7) = 13
Fibonacci(8) = 21
Fibonacci(9) = 34
Fibonacci(10) = 55


Обход конём всей шахматной доски, побывав в каждой клетке по 1 разу:

In [1]:
with cte
as
(
    select 1 as x, 1 as y, 0 as it, cast('-11' as varchar(max)) path
	union all
	select x+1, y+2, it+1, concat(path, '-', cast(x+1 as varchar), cast(y+2 as varchar) ) from cte where (x+1 between 1 and 8) and (y+2 between 1 and 8) and (charindex(concat('-', cast(x+1 as varchar), cast(y+2 as varchar) ), path) = 0) --and (it < 64)
	union all
	select x+2, y+1, it+1, concat(path, '-', cast(x+2 as varchar), cast(y+1 as varchar) ) from cte where (x+2 between 1 and 8) and (y+1 between 1 and 8) and (charindex(concat('-', cast(x+2 as varchar), cast(y+1 as varchar) ), path) = 0) --and (it < 64)
	union all
	select x+2, y-1, it+1, concat(path, '-', cast(x+2 as varchar), cast(y-1 as varchar) ) from cte where (x+2 between 1 and 8) and (y-1 between 1 and 8) and (charindex(concat('-', cast(x+2 as varchar), cast(y-1 as varchar) ), path) = 0) --and (it < 64)
	union all
	select x+1, y-2, it+1, concat(path, '-', cast(x+1 as varchar), cast(y-2 as varchar) ) from cte where (x+1 between 1 and 8) and (y-2 between 1 and 8) and (charindex(concat('-', cast(x+1 as varchar), cast(y-2 as varchar) ), path) = 0) --and (it < 64)
	union all
	select x-1, y-2, it+1, concat(path, '-', cast(x-1 as varchar), cast(y-2 as varchar) ) from cte where (x-1 between 1 and 8) and (y-2 between 1 and 8) and (charindex(concat('-', cast(x-1 as varchar), cast(y-2 as varchar) ), path) = 0) --and (it < 64)
	union all
	select x-2, y-1, it+1, concat(path, '-', cast(x-2 as varchar), cast(y-1 as varchar) ) from cte where (x-2 between 1 and 8) and (y-1 between 1 and 8) and (charindex(concat('-', cast(x-2 as varchar), cast(y-1 as varchar) ), path) = 0) --and (it < 64)
	union all
	select x-2, y+1, it+1, concat(path, '-', cast(x-2 as varchar), cast(y+1 as varchar) ) from cte where (x-2 between 1 and 8) and (y+1 between 1 and 8) and (charindex(concat('-', cast(x-2 as varchar), cast(y+1 as varchar) ), path) = 0) --and (it < 64)
	union all
	select x-1, y+2, it+1, concat(path, '-', cast(x-1 as varchar), cast(y+2 as varchar) ) from cte where (x-1 between 1 and 8) and (y+2 between 1 and 8) and (charindex(concat('-', cast(x-1 as varchar), cast(y+2 as varchar) ), path) = 0) --and (it < 64)
)
select top 1 * from cte
where it = 63

(1 row affected)

Total execution time: 00:58:49.1435336

x,y,it,path
1,4,63,-11-32-24-16-35-27-15-23-31-12-33-25-17-36-28-47-26-18-37-45-53-34-13-21-42-61-82-74-66-58-46-38-57-78-86-65-84-72-51-43-55-63-44-52-71-83-64-85-77-56-48-67-88-76-68-87-75-54-73-81-62-41-22-14


Процедура для нахождения кратчайшего пути конем из точки (x0, y0) в точку (xn, yn):

In [1]:
create or alter procedure usp_getOptimalPath(@x0 tinyint, @y0 tinyint, @xn tinyint, @yn tinyint)
as
begin
	;with cte
	as
	(
		select @x0 as x, @y0 as y, cast(0 as tinyint) as it, cast(('-'+cast(@x0 as varchar)+cast(@y0 as varchar)) as varchar(max)) path
		union all
		select cast(x+1 as tinyint), cast(y+2 as tinyint), cast(it+1 as tinyint), cast(concat(path, '-', cast(x+1 as varchar), cast(y+2 as varchar)) as varchar(max)) from cte where (x+1 between 1 and 8) and (y+2 between 1 and 8) and (charindex(concat('-', cast(x+1 as varchar), cast(y+2 as varchar) ), path) = 0) and (it < 6)
		union all
		select cast(x+2 as tinyint), cast(y+1 as tinyint), cast(it+1 as tinyint), cast(concat(path, '-', cast(x+2 as varchar), cast(y+1 as varchar)) as varchar(max)) from cte where (x+2 between 1 and 8) and (y+1 between 1 and 8) and (charindex(concat('-', cast(x+2 as varchar), cast(y+1 as varchar) ), path) = 0) and (it < 6)
		union all
		select cast(x+2 as tinyint), cast(y-1 as tinyint), cast(it+1 as tinyint), cast(concat(path, '-', cast(x+2 as varchar), cast(y-1 as varchar)) as varchar(max)) from cte where (x+2 between 1 and 8) and (y-1 between 1 and 8) and (charindex(concat('-', cast(x+2 as varchar), cast(y-1 as varchar) ), path) = 0) and (it < 6)
		union all
		select cast(x+1 as tinyint), cast(y-2 as tinyint), cast(it+1 as tinyint), cast(concat(path, '-', cast(x+1 as varchar), cast(y-2 as varchar)) as varchar(max)) from cte where (x+1 between 1 and 8) and (y-2 between 1 and 8) and (charindex(concat('-', cast(x+1 as varchar), cast(y-2 as varchar) ), path) = 0) and (it < 6)
		union all
		select cast(x-1 as tinyint), cast(y-2 as tinyint), cast(it+1 as tinyint), cast(concat(path, '-', cast(x-1 as varchar), cast(y-2 as varchar)) as varchar(max)) from cte where (x-1 between 1 and 8) and (y-2 between 1 and 8) and (charindex(concat('-', cast(x-1 as varchar), cast(y-2 as varchar) ), path) = 0) and (it < 6)
		union all
		select cast(x-2 as tinyint), cast(y-1 as tinyint), cast(it+1 as tinyint), cast(concat(path, '-', cast(x-2 as varchar), cast(y-1 as varchar)) as varchar(max)) from cte where (x-2 between 1 and 8) and (y-1 between 1 and 8) and (charindex(concat('-', cast(x-2 as varchar), cast(y-1 as varchar) ), path) = 0) and (it < 6)
		union all
		select cast(x-2 as tinyint), cast(y+1 as tinyint), cast(it+1 as tinyint), cast(concat(path, '-', cast(x-2 as varchar), cast(y+1 as varchar)) as varchar(max)) from cte where (x-2 between 1 and 8) and (y+1 between 1 and 8) and (charindex(concat('-', cast(x-2 as varchar), cast(y+1 as varchar) ), path) = 0) and (it < 6)
		union all
		select cast(x-1 as tinyint), cast(y+2 as tinyint), cast(it+1 as tinyint), cast(concat(path, '-', cast(x-1 as varchar), cast(y+2 as varchar)) as varchar(max)) from cte where (x-1 between 1 and 8) and (y+2 between 1 and 8) and (charindex(concat('-', cast(x-1 as varchar), cast(y+2 as varchar) ), path) = 0) and (it < 6)
	),
	cte_with_rn
	as
	(
		select *, row_number() over(order by len(path)) rn
		from cte
		where x=@xn and y=@yn
	)
	select * from cte_with_rn
	where rn = 1
end
go

exec usp_getOptimalPath 4, 4, 5, 5

Commands completed successfully.

Total execution time: 00:00:00.0059261

(1 row affected)

Total execution time: 00:00:01.2464618

x,y,it,path,rn
5,5,2,-44-36-55,1
